### Introduction 

A drawdown is a measurement of downside risk, and calculated as decline from an assets peak value to its lowest point (trough) over a period of time. The drawdown is usually expressed as a percentage from top to bottom.



#### Loading file into dataframe.

In [23]:
import pandas as pd 
import numpy as np
df = pd.read_csv("data_drawdown.csv", parse_dates=['date']) 
print (df)

        date  value
0 2019-01-20    180
1 2019-02-13    240
2 2019-04-04    200
3 2019-04-23    150
4 2019-05-23    350
5 2019-06-23    210
6 2019-07-23    300
7 2019-08-23    310
8 2019-09-23    200
9 2019-10-23    250



#### Using Bokeh to get visual of asset value overtime

In [24]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool

p = figure(title = 'Apple',x_axis_type = 'datetime', plot_height=200, plot_width=950)
cds_obj = ColumnDataSource(df)
p.line(x= 'date', y= 'value', source = cds_obj)
tooltips = [ ('Index','@index'),('Date','@date{%F}'),('Value', '@value')]
formatters={'date' : 'datetime'}
p.add_tools(HoverTool(tooltips=tooltips, formatters=formatters))

output_notebook()
show(p)

print (df)

Loading BokehJS ...

        date  value
0 2019-01-20    180
1 2019-02-13    240
2 2019-04-04    200
3 2019-04-23    150
4 2019-05-23    350
5 2019-06-23    210
6 2019-07-23    300
7 2019-08-23    310
8 2019-09-23    200
9 2019-10-23    250


#### Identifying peaks and troughs.

In [25]:
df['isPeak'] = np.logical_and(df.value > df.value.shift(1),df.value > df.value.shift(-1))
df['isTrough'] = np.logical_and(df.value < df.value.shift(1),df.value < df.value.shift(-1))
df['isInflection'] =  np.logical_or(df['isPeak'],df['isTrough'])
df['isInflection'].iloc[0] = True 
df['isInflection'].iloc[-1] = True
print (df)

        date  value  isPeak  isTrough  isInflection
0 2019-01-20    180   False     False          True
1 2019-02-13    240    True     False          True
2 2019-04-04    200   False     False         False
3 2019-04-23    150   False      True          True
4 2019-05-23    350    True     False          True
5 2019-06-23    210   False      True          True
6 2019-07-23    300   False     False         False
7 2019-08-23    310    True     False          True
8 2019-09-23    200   False      True          True
9 2019-10-23    250   False     False          True


#### Calculating drawdown values.

In [26]:
df = df [df['isInflection']==True]
df['drawdown_value'] = df.value - df.value.shift(1)
df['drawdown_prcnt'] = df['drawdown_value']/df.value.shift(1)
print (df)

        date  value  isPeak  isTrough  isInflection  drawdown_value  \
0 2019-01-20    180   False     False          True             NaN   
1 2019-02-13    240    True     False          True            60.0   
3 2019-04-23    150   False      True          True           -90.0   
4 2019-05-23    350    True     False          True           200.0   
5 2019-06-23    210   False      True          True          -140.0   
7 2019-08-23    310    True     False          True           100.0   
8 2019-09-23    200   False      True          True          -110.0   
9 2019-10-23    250   False     False          True            50.0   

   drawdown_prcnt  
0             NaN  
1        0.333333  
3       -0.375000  
4        1.333333  
5       -0.400000  
7        0.476190  
8       -0.354839  
9        0.250000  


#### Start and Stop date of drawdowns

In [27]:
df['drawdown_strt'] = df.date.shift(1)
df['drawdown_stop'] = df.date
print (df)

        date  value  isPeak  isTrough  isInflection  drawdown_value  \
0 2019-01-20    180   False     False          True             NaN   
1 2019-02-13    240    True     False          True            60.0   
3 2019-04-23    150   False      True          True           -90.0   
4 2019-05-23    350    True     False          True           200.0   
5 2019-06-23    210   False      True          True          -140.0   
7 2019-08-23    310    True     False          True           100.0   
8 2019-09-23    200   False      True          True          -110.0   
9 2019-10-23    250   False     False          True            50.0   

   drawdown_prcnt drawdown_strt drawdown_stop  
0             NaN           NaT    2019-01-20  
1        0.333333    2019-01-20    2019-02-13  
3       -0.375000    2019-02-13    2019-04-23  
4        1.333333    2019-04-23    2019-05-23  
5       -0.400000    2019-05-23    2019-06-23  
7        0.476190    2019-06-23    2019-08-23  
8       -0.354839    201

#### Filtering out drawdowns

In [28]:
df = df[df.drawdown_value < 0]
print (df)

        date  value  isPeak  isTrough  isInflection  drawdown_value  \
3 2019-04-23    150   False      True          True           -90.0   
5 2019-06-23    210   False      True          True          -140.0   
8 2019-09-23    200   False      True          True          -110.0   

   drawdown_prcnt drawdown_strt drawdown_stop  
3       -0.375000    2019-02-13    2019-04-23  
5       -0.400000    2019-05-23    2019-06-23  
8       -0.354839    2019-08-23    2019-09-23  
